In [3]:
import torch
import torch.nn as nn

In [2]:
!pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 14.2 MB/s eta 0:00:00


In [4]:
input = torch.randn(3,22,15000)
input.shape

torch.Size([3, 22, 15000])

In [51]:
nn.Conv1d(in_channels=22,out_channels=32,kernel_size=2,stride=2,padding=0)

Conv1d(22, 32, kernel_size=(2,), stride=(2,))

In [53]:
class Block(nn.Module):
  def __init__(self,inplace):
    super().__init__()
    self.conv1 = nn.Conv1d(in_channels=inplace,out_channels=32,kernel_size=2,stride=2,padding=0)
    self.conv2 = nn.Conv1d(in_channels=inplace,out_channels=32,kernel_size=4,stride=2,padding=1)
    self.conv3 = nn.Conv1d(in_channels=inplace,out_channels=32,kernel_size=8,stride=2,padding=3)
    self.relu = nn.ReLU()
  def forward(self,x):
    print(x.shape)
    x1 = self.conv1(x)
    x2 = self.conv2(x)
    x3 = self.conv3(x)
    print(x1.shape,x2.shape,x3.shape)
    x = torch.cat([x1,x2,x3],dim=1)
    return x

In [54]:
block = Block(22)

In [55]:
out1 = block(input)

torch.Size([3, 22, 15000])
torch.Size([3, 32, 7500]) torch.Size([3, 32, 7500]) torch.Size([3, 32, 7500])


In [56]:
out1.shape

torch.Size([3, 96, 7500])

In [57]:
block = Block(96)
out2 = block(out1)
out2.shape

torch.Size([3, 96, 7500])
torch.Size([3, 32, 3750]) torch.Size([3, 32, 3750]) torch.Size([3, 32, 3750])


torch.Size([3, 96, 3750])

In [58]:
block = Block(96)
out3 = block(out2)
out3.shape

torch.Size([3, 96, 3750])
torch.Size([3, 32, 1875]) torch.Size([3, 32, 1875]) torch.Size([3, 32, 1875])


torch.Size([3, 96, 1875])

In [59]:
gru1 = nn.GRU(input_size = 96,hidden_size = 32,batch_first = True)
gru2 = nn.GRU(input_size = 32,hidden_size = 32,batch_first = True)

x = out3.permute(0,2,1)
print(x.shape)

torch.Size([3, 1875, 96])


In [60]:
output,hn = gru(x)
print(output.shape,hn.shape)

torch.Size([3, 1875, 32]) torch.Size([1, 3, 32])


In [61]:
gru_ou1,_ = gru1(x)
gru_ou2,_ = gru2(gru_ou1)
gru_out = torch.cat([gru_ou1,gru_ou2],dim = 2)

In [62]:
gru_out.shape

torch.Size([3, 1875, 64])

In [63]:
gru3 = nn.GRU(input_size = 64,hidden_size=32,batch_first=True)
gru_out3,_ = gru3(gru_out)
gru_out3.shape

torch.Size([3, 1875, 32])

In [64]:
gru_out = torch.cat([gru_ou1,gru_ou2,gru_out3],dim=2)
gru_out.shape

torch.Size([3, 1875, 96])

In [65]:
gru4 = nn.GRU(input_size = 96,hidden_size=32,batch_first=True)
gru_out4,_ = gru4(gru_out)
gru_out4.shape

torch.Size([3, 1875, 32])

In [66]:
linear = nn.Linear(1875,1)
linear_out = linear(gru_out.permute(0,2,1))
linear_out.shape

torch.Size([3, 96, 1])

In [67]:
gru4 = nn.GRU(input_size = 96,hidden_size=32,batch_first=True)
gru_out4,_ = gru4(linear_out.permute(0,2,1))
gru_out4.shape

torch.Size([3, 1, 32])

In [101]:
input=torch.randn(3,22,15000)
input.shape

torch.Size([3, 22, 15000])

In [100]:
class Block(nn.Module):
  def __init__(self,inplace):
    super().__init__()
    self.conv1=nn.Conv1d(in_channels=inplace,out_channels=32,kernel_size=2,stride=2,padding=0)
    self.conv2=nn.Conv1d(in_channels=inplace,out_channels=32,kernel_size=4,stride=2,padding=1)
    self.conv3=nn.Conv1d(in_channels=inplace,out_channels=32,kernel_size=8,stride=2,padding=3)
    self.relu=nn.ReLU()

  def forward(self,x):
    x1=self.relu(self.conv1(x))
    x2=self.relu(self.conv2(x))
    x3=self.relu(self.conv3(x))
    x=torch.cat([x1,x3,x3],dim=1)
    return x

In [102]:
class ChronoNet(nn.Module):
  def __init__(self,channel):
    super().__init__()
    self.block1=Block(channel)
    self.block2=Block(96)
    self.block3=Block(96)
    self.gru1=nn.GRU(input_size=96,hidden_size=32,batch_first=True)
    self.gru2=nn.GRU(input_size=32,hidden_size=32,batch_first=True)
    self.gru3=nn.GRU(input_size=64,hidden_size=32,batch_first=True)
    self.gru4=nn.GRU(input_size=96,hidden_size=32,batch_first=True)
    self.gru_linear=nn.Linear(64,1)
    self.flatten=nn.Flatten()
    self.fc1=nn.Linear(32,1)
    self.relu=nn.ReLU()
  def forward(self,x):
    x=self.block1(x)
    x=self.block2(x)
    x=self.block3(x)
    x=x.permute(0,2,1)
    gru_out1,_=self.gru1(x)
    gru_out2,_=self.gru2(gru_out1)
    gru_out=torch.cat([gru_out1,gru_out2],dim=2)
    gru_out3,_=self.gru3(gru_out)
    gru_out=torch.cat([gru_out1,gru_out2,gru_out3],dim=2)
    #print('gru_out',gru_out.shape)
    linear_out=self.relu(self.gru_linear(gru_out.permute(0,2,1)))
    gru_out4,_=self.gru4(linear_out.permute(0,2,1))
    x=self.flatten(gru_out4)
    x=self.fc1(x)
    return x

In [103]:
model = ChronoNet(14)

In [104]:
model

ChronoNet(
  (block1): Block(
    (conv1): Conv1d(14, 32, kernel_size=(2,), stride=(2,))
    (conv2): Conv1d(14, 32, kernel_size=(4,), stride=(2,), padding=(1,))
    (conv3): Conv1d(14, 32, kernel_size=(8,), stride=(2,), padding=(3,))
    (relu): ReLU()
  )
  (block2): Block(
    (conv1): Conv1d(96, 32, kernel_size=(2,), stride=(2,))
    (conv2): Conv1d(96, 32, kernel_size=(4,), stride=(2,), padding=(1,))
    (conv3): Conv1d(96, 32, kernel_size=(8,), stride=(2,), padding=(3,))
    (relu): ReLU()
  )
  (block3): Block(
    (conv1): Conv1d(96, 32, kernel_size=(2,), stride=(2,))
    (conv2): Conv1d(96, 32, kernel_size=(4,), stride=(2,), padding=(1,))
    (conv3): Conv1d(96, 32, kernel_size=(8,), stride=(2,), padding=(3,))
    (relu): ReLU()
  )
  (gru1): GRU(96, 32, batch_first=True)
  (gru2): GRU(32, 32, batch_first=True)
  (gru3): GRU(64, 32, batch_first=True)
  (gru4): GRU(96, 32, batch_first=True)
  (gru_linear): Linear(in_features=64, out_features=1, bias=True)
  (flatten): Flatten(s

In [106]:
input=torch.randn(3,14,512)
input.shape

torch.Size([3, 14, 512])

In [107]:
out = model(input)

In [108]:
out.shape

torch.Size([3, 1])